In [193]:
from math import *
import numpy as np

def traverse(node,string=''):
    if isinstance(node, bin_tree):
        node=node.get_root()  
        # if unary
    if node.num_children == 1:
        string += (node.op)
        if node.coeff == None:
            string += ('('+traverse(node.left)+')')
        else:    
            string += ('('+str(node.coeff)+'*'+traverse(node.left)+')')
    # if binary
    elif node.num_children==2:
        string += '('+traverse(node.left)
        string += node.op
        string += traverse(node.right)+')'
    
    # if leaf node
    else:    
        if node.coeff == None:
            string += str(node.value)
        # if constant leaf node, will only have coefficient
        elif node.value == None:
            string += str(node.coeff)
        else:
            string += '('+str(node.coeff) + '*' + str(node.value)+')'
    return string

In [194]:
# intermediate_operands = ['sin', 'cos', 'exp', 'np.log', '+', '*']
# unary_operands = ['sin', 'cos', 'np.log', 'exp']
unary_operands = ['sin', 'cos', 'np.log']
binary_operands = ['*', '/', '+', '-', '**']
# terminal_operands = ["var1", "1"]
terminal_operands = ["c"]
coeff_max = 3



In [195]:
class PrintTree:
    """source: https://ebadahmadzadeh.wordpress.com/2014/02/26/how-to-print-a-dictionary-based-tree-in-python/"""
    def printTree(self, tree, d = 0):
        if (tree.num_children == 0):
            print("\t" * d, tree.index, tree.value)
        else:
            print( "\t" * d, tree.index, tree.name, tree.op)
            for i in [tree.left, tree.right]:
                if i != None:
                    self.printTree(i, d+1)
#                 else:
#                     print ("\t" * d, str('(') + "f" + str(')'))

In [196]:
for i in range(100):
    n1 = node(None, 1, "root")
    p = PrintTree()
    generate_tree(n1, [0.2, 0.4, 0.4], 0.08)
#     eval(traverse(n1))
    try:
        eval(traverse(n1))
    except:
        print(n1)
        print(traverse(n1))
        break
#     print(n1)
#     print(traverse(n1))
#     print(p.printTree(n1))
#     print(n1)

TypeError: __init__() takes from 2 to 3 positional arguments but 4 were given

In [197]:
import copy
class bin_tree():
    def __init__(self, delta):
        self.root = node("root")
        self.delta = delta
        self.num_nodes = 0
        self.node_list = []
        self.coeff_list = []
        
    def generate_tree(self, node, c):
        """
        generates tree recursively from a starter node
        
        :param node: ...
        
        :param c: ...
        
        :param delta: ...
        
        :return: ...
        """
        self.num_nodes += 1
        node.index = self.num_nodes
        self.node_list.append(node)
        r = np.random.rand()
        #decrease p(double branch) while
        #increase p(no branch)
        c = copy.deepcopy(c)
        c[0] += self.delta
        c[1] -= self.delta/2
        c[2] -= self.delta/2
        # no operands
        if r < c[0]:
#             node.coeff = np.random.uniform(coeff_max)
            val = np.random.choice(terminal_operands)
            # If a constant is chosen as a leaf node, then the c {index of node} is created
            if val == "c":    
                node.value = None
                node.coeff = val + str(node.index)
            else:
                node.value = val
        # unary operation
        elif r > c[0] and (r - c[0]) < c[1]:
            node.init_left()
            node.op = np.random.choice(unary_operands)
#             node.coeff = np.random.uniform(coeff_max)
            self.generate_tree(node.left, c)
        # binary
        else:
            node.init_left()
            node.init_right()
            node.op = np.random.choice(binary_operands)
#             node.coeff = 1
            self.generate_tree(node.left, c) 
            self.generate_tree(node.right, c)
            
    def get_root(self):
        return self.root 
            
    def __repr__(self):
        return traverse(self.root)
    
    def print_tree(self):
        pass
    
    def del_subtree(self):
        pass
    
    def replace_subtree_at_node(self, node):
        c = node.c
        self.del_subtree(node)
        self.generate_tree(node, c, self.delta)        

    def traverse(self):
        return traverse(self)
    
    #list of coeffs
    # delete node function
    # maintain s ame coeffs for c when deleting node

In [198]:
class node():
    
#     def __init__(self, index, name, parent=None):
#         self.name = name
#         self.num_children=0
#         self.index=index
#         self._left=None
#         self._right=None
#         self._op=None
#         self.coeff=None
#         self._value = None
#         self.parent = parent
        
#     def init_left(self, index):
#         left_node = node(index, "left", parent=self)
#         self.left = left_node
#         self.num_children += 1
        
#     def init_right(self, index):
#         right_node = node(index, "right", parent=self)
#         self.right = right_node
#         self.num_children += 1 


    def __init__(self, name, parent=None):
        self.name = name
        self.num_children=0
        self.index=None
        self._left=None
        self._right=None
        self._op=None
        self.coeff=None
        self._value = None
        self.parent = parent
        
    def init_left(self):
        left_node = node("left", parent=self)
        self.left = left_node
        self.num_children += 1
        
    def init_right(self):
        right_node = node("right", parent=self)
        self.right = right_node
        self.num_children += 1 
    
    @property    
    def left(self):
        return self._left
    
    @property    
    def right(self):
        return self._right
    
    @property    
    def op(self):
        return self._op
    
    @property    
    def value(self):
        return self._value
    
    @value.setter
    def value(self, val):
        self._value = val
    
    @left.setter
    def left(self, node):
        self._left = node
    
    @right.setter
    def right(self, node):
        self._right = node

    @op.setter
    def op(self, op):
        self._op = op
    
    def __repr__(self):
        if self.num_children > 0:
            return "(left: {}, op: {}, right: {})".format(
                self.left, self.op, self.right)
        return "{}*{}".format(self.coeff, self.value)

In [199]:
a = bin_tree(delta=0.04)
c=[0.1, 0.2]
c.append(1-sum(c))
a.generate_tree(a.get_root(), c)
p = PrintTree()
a.traverse()
# eval(a.traverse())


'(((np.log(c5)-c6)*(c8+c9))+((sin(c13)+c14)+c15))'

In [200]:
p.printTree(a.get_root())

 1 root +
	 2 left *
		 3 left -
			 4 left np.log
				 5 None
			 6 None
		 7 right +
			 8 None
			 9 None
	 10 right +
		 11 left +
			 12 left sin
				 13 None
			 14 None
		 15 None


In [48]:
traverse(a.get_root())

'np.log(1.3079251851881109*(((2.9068008159490244*1))-((1.6972356440368623*1)))+(((((1.5747441651853291*1))*(((((1.891625662694194*1))+((1.1791238349535045*1)))/((1.0628107333256887*1)))-((2.783326296299999*1))))-(((1.4200188161809952*1))*((2.3396781407008342*1))))**((1.7706431869327073*1))))'

In [39]:
a.node_list[0]

(left: (left: (left: (left: (left: 1.392786882654239*1, op: *, right: 1.6168369512562422*1), op: *, right: 1.566165858285118*1), op: -, right: 1.454028636362094*1), op: sin, right: None), op: /, right: (left: 2.641959619522911*1, op: sin, right: None))

In [24]:
len(a.node_list)

11

In [25]:
# def traverse_index(tree, a):
#     tree.index = a
#     if tree.num_children == 1:
#         tree.index = a
#         a += 1
#         traverse_index(tree.left, a)
#     elif tree.num_children==2:
#         tree.index = a
#         a += 1
#         traverse_index(tree.left, a)
#         a += 1
#         traverse_index(tree.right, a)

        

In [26]:
# g = bin_tree()

# g.generate_tree(g.get_root(), [0.2, 0.4, 0.4], 0.04)

# a=1
# s=1
# traverse_index(a.get_root(), s)
# p.printTree(a.get_root())

 1 root /
	 2 left sin
		 3 left -
			 4 left *
				 5 left *
					 6 1
					 7 1
				 6 1
			 5 1
	 3 right sin
		 4 1
